In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as pof
import scipy.stats as sps
sns.set()

plt.rc('font', size=30)
plt.rc('axes', titlesize=30)
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=30)
plt.rc('ytick', labelsize=30)
plt.rc('legend', fontsize=30)
plt.rc('figure', titlesize=30)

from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.covariance import MinCovDet
from tqdm import tqdm
import cvxpy as cvx

In [5]:
%run new_portfolio_optimizer.ipynb

In [2]:
def add_row_in_summary_table(names, optimizers, risk_free_return, df_train, df_test, compare_table):
    for name, optimizer in tqdm(zip(names, optimizers)):
        best_params = optimizer.best_params_
        po = optimizer.best_estimator_
        po.refit(df_train)
        predictions = po.predict(df_test)
        final_return = predictions[1][-1]
        sharp_score = po.set_params(period_for_pred=120).score(df_test, how='sharp')
        sortino_score = po.score(df_test, how='sortino')
        var_score = po.set_params(period_for_pred=30).score(df_test, how='VAR')
        ave_block_return = np.mean(predictions[0])


        name_method = {'method': name}
        statistics = {'sharp_score': sharp_score, 'sortino_score': sortino_score, 'VaR_score' : var_score,
                      'final_return': final_return, 'ave_block_return': ave_block_return}
        row = dict(name_method)
        row.update(best_params)
        row.update(statistics)


        if compare_table is None:
            compare_table = pd.DataFrame([row])
        else:
            if name_method in list(compare_table['method']):
                compare_table = compare_table[compare_table['method'] != name_method]

            compare_table = pd.concat([compare_table, pd.DataFrame([row])])
            compare_table.index = np.arange(len(compare_table))
    return compare_table

In [3]:
def plot_results(names, optimizers, df_train, df_test, title, sp=None):
    fig = go.Figure()
    
    for name, optimizer in tqdm(zip(names, optimizers)):
        po = optimizer.best_estimator_
        po.refit(df_train)
        cum_predict = po.predict(df_test)[1]
        fig.add_trace(go.Scatter(x=cum_predict.index, y=cum_predict, name=name))
        
    if sp is not None:
        fig.add_trace(go.Scatter(x=sp.index, y=sp, name='Индекс S&P 500'))
            
    fig.update_layout(title=title, xaxis_title='time', 
                  yaxis_title='return')
    fig.show()
    pof.plot(fig, filename=f'{title}.html', auto_open=False)